In [ ]:
%matplotlib notebook

In [ ]:
import json
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import quaternion as qt
import scipy as sp

---

## Load JSONs to Memory

In [ ]:
logfile_path = pathlib.Path() / "pose.jsons"

In [ ]:
jsons = []
data = []
with open(logfile_path) as f:
    for line in f:
        jsons.append(line)
for single_json in jsons:
    data.append(json.loads(single_json))

# At this point, `data` is a list of dictionaries, each containing the poses of all the boxes at a given timestamp
# in a JSON-like format.

---

## Convert Raw Data to Box Pose Data

In [ ]:
def stamp_to_time(stamp):
    # Parse JSON time stamps to a numerical value.
    t = 0
    if 'sec' in stamp:
        t += int(stamp['sec'])
    if 'nsec' in stamp:
        t += 1e-9 * stamp['nsec']
    return t

In [ ]:
def position_to_p(position):
    # Parse JSON position values and return as an array.
    p = np.zeros(3)
    if 'x' in position:
        p[0] = position['x']
    if 'y' in position:
        p[1] = position['y']
    if 'z' in position:
        p[2] = position['z']
    return p

In [ ]:
def orientation_to_q(orientation):
    # Parse JSON quaternion values and return as an array.
    q_arr = np.zeros(4)
    if 'w' in orientation:
        q_arr[0] = orientation['w']
    if 'x' in orientation:
        q_arr[1] = orientation['x']
    if 'y' in orientation:
        q_arr[2] = orientation['y']
    if 'z' in orientation:
        q_arr[3] = orientation['z']
    return qt.from_float_array(q_arr)

In [ ]:
# Names of the boxes present in the JSON file.
names = [
    'box_xx',
    'box_xy',
    'box_xz',
    'box_xp',
    'box_xq',
    'box_xr',
    'box_yy',
    'box_yz',
    'box_yp',
    'box_yq',
    'box_yr',
    'box_zz',
    'box_zp',
    'box_zq',
    'box_zr',
    'box_pp',
    'box_pq',
    'box_pr',
    'box_qq',
    'box_qr',
    'box_rr',
]

In [ ]:
# Custom `dtype` used to store poses. Each pose will contain a timestamp 't' (in seconds), a position 'p' (in m),
# and a quaternion representing orientation 'q'.
pose_dtype = np.dtype([('t', np.float64), ('p', np.float64, 3), ('q', qt.quaternion)])

In [ ]:
# Preallocate memory.
box_poses = {}
for name in names:
    box_poses[name] = np.zeros(len(data), dtype=pose_dtype)

In [ ]:
# Convert JSON data to `pose_dtype` data.
for n, tdata in enumerate(data):
    t = stamp_to_time(tdata['header']['stamp'])
    for single_pose in tdata['pose']:
        name = single_pose['name']
        if name in names:
            box_poses[name][n]['t'] = t
            box_poses[name][n]['p'] = position_to_p(single_pose['position'])
            box_poses[name][n]['q'] = orientation_to_q(single_pose['orientation'])

# At this point, `box_poses` is a dictionary, with the box names as the keys, and an array of poses as each value.
# The array of poses contains the motion of a single box and uses `pose_dtype` as dtype.

---

## Generate Box Motion from Scratch

In [ ]:
# Body spatial inertia, expressed in body frame.
mass = 1
inertia_body = 0.1 * np.eye(3)
spatial_inertia_body = np.r_[
    np.c_[mass * np.eye(3), np.zeros((3, 3))],
    np.c_[np.zeros((3, 3)), inertia_body]]

In [ ]:
# Added mass matrix, expressed in body frame.
added_mass_body = np.zeros((6, 6))
added_mass_body[0, 1] += 0.261666
added_mass_body[1, 0] += 0.261666

In [ ]:
# Combined spatial_inertia.
full_spatial_inertia_body = spatial_inertia_body + added_mass_body

In [ ]:
# Force and torques applied throughout the motion, in the absolute reference frame.
force = np.array([1, 0, 0])
torque = np.zeros(3)

In [ ]:
# Initial position, orientation, linear and angular velocities, of the body-fixed frame.
p0 = np.array([0, 0, 0.5])
q0 = qt.one
v0 = np.zeros(3)
w0 = np.zeros(3)

In [ ]:
# Get timestamps from one of the boxes, they should be the same for all the boxes.
ts = box_poses['box_xx']['t']

In [ ]:
def skew(v):
    # Return the left cross-product matrix for vector `v`.
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def eom(pose, vels, forces):
    # Equations of motion for the rigid body. The (body-fixed) spatial inertia is taken to be
    # `spatial_inertia_body` expressed in the body frame at the initial instant of time.
    # `spatial_inertia_body` must be invertible, no other assumptions are made (e.g. no assumptions about
    # symmetry or positive-definiteness).
    v, w = vels[:3], vels[3:]
    R = qt.as_rotation_matrix(pose['q'])
    big_R = sp.linalg.block_diag(R, R)
    w_hat = skew(w)
    big_w_hat = sp.linalg.block_diag(w_hat, w_hat)
    
    # Spatial inertia (and derivatives) in the absolute frame.
    M = big_R @ full_spatial_inertia_body @ big_R.T
    M_avg = 0.5 * (M + M.T)
    M_avg_dot = big_w_hat @ M_avg - M_avg @ big_w_hat
    M_avg_inv = np.linalg.inv(M_avg)
    
    M12 = M[:3, 3:]
    M21 = M[3:, :3]
    # Torque correction due to non block-diagonal terms.
    torque_correction = 0.5 * (w_hat @ M21 - M21 @ w_hat) @ v + 0.5 * (w_hat @ M12.T - M12.T @ w_hat) @ v
    f_star = np.r_[
        forces[:3],
        forces[3:] + torque_correction]
    
    accs = M_avg_inv @ (f_star - M_avg_dot @ vels)
    
    return accs

In [ ]:
# Preallocate memory for the generated motion.
generated_poses = np.zeros(len(ts), dtype=pose_dtype)
vels = np.zeros((len(ts), 6))
accs = np.zeros((len(ts) - 1, 6))

In [ ]:
# Integrate motion from the starting conditions. Use Euler's method for velocity, and a quadratic integrator for
# position/orientation. This is expected to be inaccurate, but should give an idea of how the motion looks like.
generated_poses[0]['t'] = ts[0]
generated_poses[0]['p'] = p0
generated_poses[0]['q'] = q0
vels[0, :] = np.r_[v0, w0]
forces = np.r_[force, torque]
for n in range(len(ts) - 1):
    dt = ts[n + 1] - ts[n]
    accs[n] = eom(generated_poses[n], vels[n, :], forces)
    vels[n + 1] = vels[n] + accs[n] * dt
    generated_poses[n + 1]['t'] = ts[n + 1]
    generated_poses[n + 1]['p'] = (
        generated_poses[n]['p'] +
        vels[n, :3] * dt +
        0.5 * accs[n, :3] * dt ** 2
    )
    generated_poses[n + 1]['q'] = (
        qt.from_rotation_vector(
            vels[n, 3:] * dt +
            0.5 * accs[n, 3:] * dt ** 2
        ) *
        generated_poses[n]['q']
    )

# At this point, `generated_poses` contains the generated motion in the form of an array of `pose_dtype` dtype;
# `vels` is an array with the generated linear and angular velocities as 6-vectors (in this order); and `accs`
# contains the generated linear and angular accelerations as 6-vectors (in this order).

---

## Compare Motions

In [ ]:
# Which box to compare the generated motion against.
box_name = 'box_xy'

In [ ]:
fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(box_poses[box_name]['t'], box_poses[box_name]['p'][:, 0])
plt.plot(generated_poses['t'], generated_poses['p'][:, 0])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(box_poses[box_name]['t'], box_poses[box_name]['p'][:, 1])
plt.plot(generated_poses['t'], generated_poses['p'][:, 1])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(box_poses[box_name]['t'], box_poses[box_name]['p'][:, 2])
plt.plot(generated_poses['t'], generated_poses['p'][:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Position')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
fig = plt.figure()

plt.subplot(2, 2, 1)
plt.plot(box_poses[box_name]['t'], qt.as_float_array(box_poses[box_name]['q'])[:, 0])
plt.plot(generated_poses['t'], qt.as_float_array(generated_poses['q'])[:, 0])
plt.grid()
plt.ylabel('w')

plt.subplot(2, 2, 2)
plt.plot(box_poses[box_name]['t'], qt.as_float_array(box_poses[box_name]['q'])[:, 1])
plt.plot(generated_poses['t'], qt.as_float_array(generated_poses['q'])[:, 1])
plt.grid()
plt.ylabel('x')

plt.subplot(2, 2, 3)
plt.plot(box_poses[box_name]['t'], qt.as_float_array(box_poses[box_name]['q'])[:, 2])
plt.plot(generated_poses['t'], qt.as_float_array(generated_poses['q'])[:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('y')

plt.subplot(2, 2, 4)
plt.plot(box_poses[box_name]['t'], qt.as_float_array(box_poses[box_name]['q'])[:, 3])
plt.plot(generated_poses['t'], qt.as_float_array(generated_poses['q'])[:, 3])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Quaternion')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
# Compute box velocities.
box_vels = np.zeros((box_poses[box_name].shape[0] - 1, 6))
for n in range(box_poses[box_name].shape[0] - 1):
    q = box_poses[box_name]['q'][n]
    dt = box_poses[box_name]['t'][n + 1] - box_poses[box_name]['t'][n]
    dp = box_poses[box_name]['p'][n + 1] - box_poses[box_name]['p'][n]
    dq = box_poses[box_name]['q'][n + 1] - q
    v = dp / dt
    w = qt.as_float_array(2 * (dq / dt) * q.conj())[1:]
    box_vels[n, 0:3] = v
    box_vels[n, 3:6] = w

In [ ]:
fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(box_poses[box_name]['t'][:-1], box_vels[:, 0])
plt.plot(generated_poses['t'], vels[:, 0])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(box_poses[box_name]['t'][:-1], box_vels[:, 1])
plt.plot(generated_poses['t'], vels[:, 1])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(box_poses[box_name]['t'][:-1], box_vels[:, 2])
plt.plot(generated_poses['t'], vels[:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Linear Velocity')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(box_poses[box_name]['t'][:-1], box_vels[:, 3])
plt.plot(generated_poses['t'], vels[:, 3])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(box_poses[box_name]['t'][:-1], box_vels[:, 4])
plt.plot(generated_poses['t'], vels[:, 4])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(box_poses[box_name]['t'][:-1], box_vels[:, 5])
plt.plot(generated_poses['t'], vels[:, 5])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Angular Velocity')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
# Compute box accelerations.
box_accs = np.zeros((box_poses[box_name].shape[0] - 2, 6))
for n in range(box_poses[box_name].shape[0] - 2):
    dt = box_poses[box_name]['t'][n + 1] - box_poses[box_name]['t'][n]
    box_accs[n] = (box_vels[n + 1] - box_vels[n]) / dt

In [ ]:
fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(box_poses[box_name]['t'][:-2], box_accs[:, 0])
plt.plot(generated_poses['t'][:-1], accs[:, 0])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(box_poses[box_name]['t'][:-2], box_accs[:, 1])
plt.plot(generated_poses['t'][:-1], accs[:, 1])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(box_poses[box_name]['t'][:-2], box_accs[:, 2])
plt.plot(generated_poses['t'][:-1], accs[:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Linear Acceleration')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(box_poses[box_name]['t'][:-2], box_accs[:, 3])
plt.plot(generated_poses['t'][:-1], accs[:, 3])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(box_poses[box_name]['t'][:-2], box_accs[:, 4])
plt.plot(generated_poses['t'][:-1], accs[:, 4])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(box_poses[box_name]['t'][:-2], box_accs[:, 5])
plt.plot(generated_poses['t'][:-1], accs[:, 5])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Angular Acceleration')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
def compute_momenta(pose, vels):
    # Compute linear and angular momenta. Return as a 6-dimensional array.
    R = qt.as_rotation_matrix(pose['q'])
    big_R = sp.linalg.block_diag(R, R)
    M = big_R @ full_spatial_inertia_body @ big_R.T
    return M @ vels

In [ ]:
def compute_kinetic_energy(pose, vels):
    # Compute kinetic energy.
    R = qt.as_rotation_matrix(pose['q'])
    big_R = sp.linalg.block_diag(R, R)
    M = big_R @ full_spatial_inertia_body @ big_R.T
    return 0.5 * vels @ M @ vels

In [ ]:
# Compute linear/angular momentum and kinetic energy. Assume that the spatial inertia matrix used by Gazebo is the
# same one used in this notebook.
box_momenta = np.zeros((ts.shape[0] - 1, 6))
box_energy = np.zeros(ts.shape[0] - 1)
momenta = np.zeros((ts.shape[0], 6))
energy = np.zeros(ts.shape[0])
for n in range(ts.shape[0] - 1):
    box_momenta[n, :] = compute_momenta(box_poses[box_name][n], box_vels[n, :])
    box_energy[n] = compute_kinetic_energy(box_poses[box_name][n], box_vels[n, :])
for n in range(ts.shape[0]):
    momenta[n, :] = compute_momenta(generated_poses[n], vels[n, :])
    energy[n] = compute_kinetic_energy(generated_poses[n], vels[n, :])

In [ ]:
fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(box_poses[box_name]['t'][:-1], box_momenta[:, 0])
plt.plot(generated_poses['t'], momenta[:, 0])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(box_poses[box_name]['t'][:-1], box_momenta[:, 1])
plt.plot(generated_poses['t'], momenta[:, 1])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(box_poses[box_name]['t'][:-1], box_momenta[:, 2])
plt.plot(generated_poses['t'], momenta[:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Linear Momentum')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(box_poses[box_name]['t'][:-1], box_momenta[:, 3])
plt.plot(generated_poses['t'], momenta[:, 3])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(box_poses[box_name]['t'][:-1], box_momenta[:, 4])
plt.plot(generated_poses['t'], momenta[:, 4])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(box_poses[box_name]['t'][:-1], box_momenta[:, 5])
plt.plot(generated_poses['t'], momenta[:, 5])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Angular Momentum')
plt.figlegend(['Gazebo', 'Notebook'])

In [ ]:
fig = plt.figure()

plt.plot(box_poses[box_name]['t'][:-1], box_energy)
plt.plot(generated_poses['t'], energy)
plt.grid()
plt.xlabel('time')
plt.ylabel('T')

plt.title('Kinetic Energy')
plt.legend(['Gazebo', 'Notebook'])